The discrete convolution:
$$
\partial_t^\alpha (t_n) = \frac{1}{\tau^\alpha} \sum_{j=1}^{n+1} \omega_j^\alpha \Big[u(t_{n-j+1}) - u(0)  \Big]
$$
with 
<!-- $$ -->
<!-- \omega_0^\alpha = 1; \quad \omega_j^\alpha = - \frac{\alpha -j + 1}{j}, \quad j=1,\ldots . -->
<!-- $$ -->

$$
\omega_1^\alpha = 1; \quad \omega_j^\alpha = \frac{j-α-2}{j-1} ω_{j-1}^α, \quad j=2,\ldots .
$$

The following scheme is implemented:

$$
 \displaystyle \left(\frac{1}{τ^α} ω_1 M + A\right)u^n = f^n + \frac{1}{τ^α} \sum_{j=1}^{n} ω_j M u^1 - \frac{1}{τ^α} \sum_{j=2}^{n} ω_{n-j+1} M u^{j}
$$

The fokker plank equation:
$$
u - \nabla \cdot(^R∂_t^{1-α} κ_α ∇u - \boldsymbol{F} ^R∂_t^{1-α }u) = f \qquad \text{in} \quad Ω × (0,T]
$$
subject to the initial and boundary conditions. This can be further simplified into 
$$
\partial_t^α u - κ_α Δu + ∇⋅κ_α ∇u+ \nabla \cdot( \boldsymbol{F} u) = f 
$$

Introducing $\boldsymbol{σ} = -κ_α \nabla u + {\bf F} u$ so that

$$
κ_α^{-1} \boldsymbol{σ} + ∇u = κ_α^{-1}{\bf F}u.
$$

The weak/discrete form is:
$$
\left(∂_t^α u, v\right) + \left(\nabla \cdot \boldsymbol{σ}, v\right) = \left(f,v\right) \qquad ∀ v\in V \\ 
\left(κ^{-1} \boldsymbol{σ}, \boldsymbol{\tau}\right) - \left(u, ∇⋅\boldsymbol{\tau}\right) -\left(κ_α^{-1}{\bf F}u,\boldsymbol{\tau}\right) =0 \qquad \forall \boldsymbol{\tau} \in {\bf W}
$$

In [73]:
module FractionalFokkerMixedConvQuad

using LinearAlgebra
using Printf
using GradientRobustMultiPhysics
using ExtendableGrids
using GridVisualize
using ExtendableSparse
using SpecialFunctions

"""
    get_problem_data(ν)

TBW
"""
function get_problem_data(ϵ, p=1, α=1.0)
    γ= DataFunction([0.0]; name = "γ")    
    function coeffbeta!(result, x, t)
        result[1] = x[1];
        result[2] = x[2];
    end    
    β = DataFunction(coeffbeta!, [2,2]; name="β", dependencies="XT", bonus_quadorder=1)
    function exact!(result, x, t)
        result[1] = (x[1]-x[1]^2)*(x[2]-x[2]^2)*(1.0+t^p/gamma(α+1));
    end    
    u = DataFunction(exact!, [1,2]; name="u", dependencies="XT", bonus_quadorder=5)
    dt_u = eval_dt(u)
    ∇u = eval_∇(u)
    Δu = eval_Δ(u)
    function rhs!(result, x, t)    
        # dt_u = x[1]*(1-x[1])*x[2]*(1-x[2])*gamma(p+1)/gamma(p-α+1)*t^(p-α) 
        u = (x[1]-x[1]^2)*(x[2]-x[2]^2)*(1.0+t);
        ux = (1-2*x[1])*(x[2]-x[2]^2)*(1.0+t);
        uxx = -2*(x[2]-x[2]^2)*(1.0+t);
        uy = (x[1]-x[1]^2)*(1-2*x[2])*(1.0+t); 
        uyy = -2*(x[1]-x[1]^2)*(1.0+t);
        ut = (x[1]-x[1]^2)*(x[2]-x[2]^2);
        result[1] = ut - (uxx+uyy) + x[1]*ux+x[2]*uy + 2*u;
      return nothing
    end
    f = DataFunction(rhs!, [1,2]; name = "f", dependencies = "XT", bonus_quadorder = 5)  

    return γ, β, u, ∇(u), f
end

function main(;ϵ=1.0, nrefinements=2, nsteps=10, T0=0, Tf=1.0, α=1, tpower=1.0, verbosity=0)
    # generating fractional derivative data
    ω = zeros(nsteps+1)
    ω[1] = 1.0
    ω[2] = -α
    for j=3:nsteps+1
        ω[j] = (j-2-α)/(j-1)*ω[j-1]
    end
    @show ω
    # computing the sum of ω
    sum_omega = zeros(nsteps+1)
    sum_omega[1] = ω[1]
    for j=2:nsteps+1
        sum_omega[j] = sum_omega[j-1] + ω[j]
    end
    @show  sum_omega;
    
    ## finite elements and bilinear forms
    # Prepare the matrices and rhs coming from space discretization 
    ## set log level
    set_verbosity(verbosity)    
    # choose a finite element type

    FEType = [H1P1{1}, HDIVRT1{2}]
    #TODO: fix from the problem data
    u0 = DataFunction([0.0])

    ## negotiate data functions to the package
    γ, β, u, ∇u, f = get_problem_data(ϵ, tpower, α)

    # temp=div(β)
    # function div_kernel(result, input, x)
    #     # evaluate β
            
    #     eval_data!(temp)
    #     # compute β ⋅ \sigma
    #     result[1] = temp.val[1]*input[1]
    #     end
    #     action = Action(div_kernel, [1, 1]; dependencies = "X", bonus_quadorder = β.bonus_quadorder) 
    #     something = BilinearForm([Identity, Identity], action; factor = -1/ϵ)
    # end
    function convection_kernel(result, input, x, t)
        # evaluate β        
        eval_data!(β, x, t)
        # compute β ⋅ \sigma
            result[1] = 0.0
            result[2] = 0.0
        for k = 1 : 2
            result[k] += β.val[k]*input[1]
        end
    end
    action = Action(convection_kernel, [2, 1]; dependencies = "XT", bonus_quadorder = β.bonus_quadorder) 
    CO = BilinearForm([Identity, Identity], action; factor = -1/ϵ, transposed_assembly = false, apply_action_to=[2])

    ## load initial mesh
    #xgrid = grid_unitsquare(Triangle2D)
    xgrid = uniform_refine(grid_unitsquare(Triangle2D), nrefinements)
    FES = [FESpace{FEType[1]}(xgrid; broken = true), FESpace{FEType[2]}(xgrid)]
    # @show FES
    Solution = FEVector(FES)
    n_dofs = FES[1].ndofs + FES[2].ndofs
    interpolate!(Solution[1], u; time = 0.0)
    #@show Solution
    
    # matrices 
    M = FEMatrix(FES)
    assemble_operator!(M[1,1], BilinearForm([Identity, Identity]))
    # @show M.entries
    # println(size(M[1,1]))
    #println("ndofs: ", FES[1].ndofs)
    A = FEMatrix(FES)
    # assemble_operator!(A[1,2], CO; time=0.0) # (u, \tau)
    # assemble_operator!(A[1,1], ReactionOperator(γ); time=0.0) # (u,v)
    # assemble_operator!(A[2,1], LagrangeMultiplier(Divergence)) # (\sigma, v)
    # assemble_operator!(A[2,2], BilinearForm([Identity, Identity]; factor = 1/ϵ)) #(\sigma, \tau)
    # assemble_operator!(A[2,1], CO; time=0.0) #(\eps^-1 F , \tau)
    # @show A.entries

    rhs = FEVector(FES)
    # assemble_operator!(rhs[1], LinearForm(Identity, f; factor=-1); time=0.0)
    # assemble_operator!(rhs[2], LinearForm(NormalFlux, u; AT=ON_BFACES); time=0.0)
    # @show rhs.entries
    
    t0 = T0
    tau = (Tf - T0)/nsteps
    @show nrefinements, nsteps

    V1 = zeros(Float64, FES[1].ndofs+FES[2].ndofs, 1)
    #V1 = FEVector(FES)
    Mu0 = zeros(Float64, FES[1].ndofs+FES[2].ndofs)

    SystemMatrix = FEMatrix(FES)
    # @show SystemMatrix
    SystemRHS = FEVector(FES)
    SystemSol = FEVector(FES)

    SolVector = Array{FEVector{Float64}}([])
    # ndofs
    SolVector = zeros(Float64, n_dofs, nsteps)
    SolVector[:,1] = Solution[1].entries

    @show norm(Solution.entries - SolVector[:,1])
    
    # time loop
    scale = tau^α
    l2 = zero(Float64); h1 = zero(Float64)
    oldL2 = zero(Float64); oldh1 = zero(Float64)
    eL2 = zero(Float64); eh1 = zero(Float64)

    Mu0[:] = M.entries*Solution.entries
    
    @show scale
    for m = 2:nsteps
        t0 = t0 + tau        
        # assemble rhs 
        fill!(SystemRHS.entries, 0)
        fill!(rhs.entries, 0)
        assemble_operator!(rhs[1], LinearForm(Identity, f; factor = 1.0); time=t0)
        assemble_operator!(rhs[2], LinearForm(NormalFlux, u; AT=ON_BFACES, factor = -1); time=t0)
        V1[:, 1] = rhs.entries

        fill!(SystemMatrix.entries.cscmatrix.nzval, 0)
        fill!(A.entries.cscmatrix.nzval, 0)
        # assembling the matrices with coefficients depending on time     
        assemble_operator!(A[1,1], ReactionOperator(γ); time=t0)
        assemble_operator!(A[2,1], BilinearForm([Divergence, Identity]; factor = -1.0)) 
        assemble_operator!(A[1,2], BilinearForm([Identity, Divergence]; factor =  1.0))        
        assemble_operator!(A[2,2], BilinearForm([Identity, Identity]; factor = 1/ϵ))
        assemble_operator!(A[2,1], CO; time = t0)
        #@show t0, m, sum_omega[m]
        # system left hand side 1/τ^α * (M + A)u^n; ω_0= 1.0
        addblock!(SystemMatrix[1, 1], M[1, 1]; factor= ω[1]/tau)

        addblock!(SystemMatrix[1, 1], A[1, 1]; factor= 1.0)
        addblock!(SystemMatrix[1, 2], A[1, 2]; factor= 1.0)
        addblock!(SystemMatrix[2, 1], A[2, 1]; factor= 1.0)
        addblock!(SystemMatrix[2, 2], A[2, 2]; factor= 1.0)
        #@info "Preparing the rhs with right coefficients"
        # rhs at current time 
        addblock!(SystemRHS[1], V1[:,1]; factor= 1.0 )
        # 1/tau^α * ∑ ω_j^α M u^0 
        # addblock!(SystemRHS[1], Mu0; factor= sum_omega[m]/scale)
        # Mu0[:] = M.entries*Solution.entries
        addblock!(SystemRHS[1], Mu0; factor= sum_omega[m-1]/scale)
        # @show sum_omega[m-1] # please check me once again
        # 1/τ^α * ∑_j=2^{n-1} ω_{n-j+1}^ℵ M u^j 
        for j=2:m-1 
            addblock!(SystemRHS[1], M.entries*SolVector[:,j]; factor= -ω[m-j+1]/scale)
        end

        flush!(SystemMatrix.entries)      
        #@show SystemRHS.entries

        SystemSol.entries[:] = SystemMatrix.entries \ SystemRHS.entries
        for j = 1 : n_dofs 
            Solution[1][j] = SystemSol[1][j]
        end

        # push!(SolVector, Solution)
        SolVector[:,m] = Solution[1].entries

        L2Error_u = L2ErrorIntegrator(u, Identity; time= t0)
        l2 = evaluate(L2Error_u, Solution[1])
        eL2 += (l2 + oldL2) * tau * 0.5
        oldL2 = l2

        function error_kernel(result, input, x)
            # evaluate β        
            eval_data!(β, x, t0)
            eval_data!(∇u, x, t0)
            # compute β ⋅ \sigma
                result[1] = 0.0                
            for k = 1 : 2
                result[1] += (input[1+k]-β.val[k]*input[1] + ∇u.val[k])^2
            end
        end
        action = Action(error_kernel, [1, 3]; dependencies = "X", bonus_quadorder = β.bonus_quadorder) 
        h1erro = ItemIntegrator([Identity, Identity], action)
        
        h1 = evaluate(h1erro, [Solution[1], Solution[2]])
        eh1 += (h1 + oldh1) * tau * 0.5
        oldh1 = h1

        @show sqrt(l2), sqrt(h1)
    end
    @show (sqrt(l2), sqrt(h1), sqrt(eL2), sqrt(eh1))
    (sqrt(l2), sqrt(h1), sqrt(eL2), sqrt(eh1))
end

end

Main.FractionalFokkerMixedConvQuad

In [74]:
ns = [1 2 3]
eL2=[]; eH1=[];
for n in ns
    el2, eh1 = Main.FractionalFokkerMixedConvQuad.main(nrefinements=n, nsteps=3, α=1.0, tpower=1)
    push!(eL2, el2)
    push!(eH1, eh1)
end

ω = [1.0, -1.0, -0.0, -0.0]
sum_omega = [1.0, 0.0, 0.0, 0.0]


(nrefinements, nsteps) = (1, 3)
norm(Solution.entries - SolVector[:, 1]) = 0.0
scale = 0.3333333333333333
(sqrt(l2), sqrt(h1)) = 

(0.002995643559614887, 0.019939762397360616)
(sqrt(l2), sqrt(h1)) = (0.0036784000221520108, 0.024636202679542737)
(sqrt(l2), sqrt(h1), sqrt(eL2), sqrt(eh1)) = (0.0036784000221520108, 0.024636202679542737, 0.0022905016697764495, 0.015286871989792972)
ω = [1.0, -1.0, -0.0, -0.0]
sum_omega = [1.0, 0.0, 0.0, 0.0]
(nrefinements, nsteps) = (2, 3)
norm(Solution.entries - SolVector[:, 1]) = 0.0
scale = 0.3333333333333333
(sqrt(l2), sqrt(h1)) = (0.0007791309225902866, 0.004982888301123717)
(sqrt(l2), sqrt(h1)) = (0.0009455446654559189, 0.0061327691274202555)
(sqrt(l2), sqrt(h1), sqrt(eL2), sqrt(eh1)) = (0.0009455446654559189, 0.0061327691274202555, 0.0005927541232029977, 0.003813773477218029)
ω = [1.0, -1.0, -0.0, -0.0]
sum_omega = [1.0, 0.0, 0.0, 0.0]
(nrefinements, nsteps) = (3, 3)
norm(Solution.entries - SolVector[:, 1]) = 0.0
scale = 0.3333333333333333
(sqrt(l2), sqrt(h1)) = (0.00019625820641569141, 0.0012452520471360113)


(sqrt(l2), sqrt(h1)) = (0.00023748714961539056, 0.001531470149290617)
(sqrt(l2), sqrt(h1), sqrt(eL2), sqrt(eh1)) = (0.00023748714961539056, 0.001531470149290617, 0.00014912786091329547, 0.0009527771810135777)


In [67]:
log2(0.0073623182795670645/0.0018383650015769742)

2.0017368824396744